In [ ]:
%%html
<link href="http://mathbook.pugetsound.edu/beta/mathbook-content.css" rel="stylesheet" type="text/css" />
<link href="https://aimath.org/mathbook/mathbook-add-on.css" rel="stylesheet" type="text/css" />
<style>.subtitle {font-size:medium; display:block}</style>
<link href="https://fonts.googleapis.com/css?family=Open+Sans:400,400italic,600,600italic" rel="stylesheet" type="text/css" />
<link href="https://fonts.googleapis.com/css?family=Inconsolata:400,700&subset=latin,latin-ext" rel="stylesheet" type="text/css" /><!-- Hide this cell. -->
<script>
var cell = $(".container .cell").eq(0), ia = cell.find(".input_area")
if (cell.find(".toggle-button").length == 0) {
ia.after(
    $('<button class="toggle-button">Toggle hidden code</button>').click(
        function (){ ia.toggle() }
        )
    )
ia.hide()
}
</script>


**Important:** to view this notebook properly you will need to execute the cell above, which assumes you have an Internet connection.  It should already be selected, or place your cursor anywhere above to select.  Then press the "Run" button in the menu bar above (the right-pointing arrowhead), or press Shift-Enter on your keyboard.

$\newcommand{\identity}{\mathrm{id}}
\newcommand{\notdivide}{\nmid}
\newcommand{\notsubset}{\not\subset}
\newcommand{\lcm}{\operatorname{lcm}}
\newcommand{\gf}{\operatorname{GF}}
\newcommand{\inn}{\operatorname{Inn}}
\newcommand{\aut}{\operatorname{Aut}}
\newcommand{\Hom}{\operatorname{Hom}}
\newcommand{\cis}{\operatorname{cis}}
\newcommand{\chr}{\operatorname{char}}
\newcommand{\Null}{\operatorname{Null}}
\newcommand{\lt}{<}
\newcommand{\gt}{>}
\newcommand{\amp}{&}
$

<div class="mathbook-content"><h2 class="heading hide-type" alt="Section 7.6 Sage"><span class="type">Section</span><span class="codenumber">7.6</span><span class="title">Sage</span></h2><a href="crypt-sage.ipynb" class="permalink">¶</a></div>

<div class="mathbook-content"></div>

<div class="mathbook-content"><p id="p-1169">Since Sage began as software to support research in number theory, we can quickly and easily demonstrate the internal workings of the <abbr class="acronym">RSA</abbr> algorithm.  Recognize that, in practice, many other details such as encoding between letters and integers, or protecting one's private key, are equally important for the security of communications.  So <abbr class="acronym">RSA</abbr> itself is just the theoretical foundation.</p></div>

<div class="mathbook-content"><h3 class="heading hide-type" alt="Subsection  Constructing Keys"><span class="type">Subsection</span><span class="codenumber" /><span class="title">Constructing Keys</span></h3></div>

<div class="mathbook-content"><p id="p-1170">We will suppose that Alice wants to send a secret message to Bob, along with message verification (also known as a message with a digital signature).  So we begin with the construction of key pairs (private and public) for both Alice and Bob.  We first need two large primes for both individuals, and their product.  In practice, values of $n$ would have hundreds of digits, rather than just $21$ as we have done here.</p></div>

In [ ]:
p_a = next_prime(10^10)
q_a = next_prime(p_a)
p_b = next_prime((3/2)*10^10)
q_b = next_prime(p_b)
n_a = p_a * q_a
n_b = p_b * q_b
n_a, n_b

<div class="mathbook-content"><p id="p-1171">Computationally, the value of the Euler $\phi$-function for a product of primes $pq$ can be obtained from $(p-1)(q-1)\text{,}$ but we could use Sage's built-in function just as well.</p></div>

In [ ]:
m_a = euler_phi(n_a)
m_b = euler_phi(n_b)
m_a, m_b

<div class="mathbook-content"><p id="p-1172">Now we can create the encryption and decryption exponents.  We choose the encryption exponent as a (small) number relatively prime to the value of $m\text{.}$  With Sage we can factor $m$ quickly to help us choose this value.  In practice we would not want to do this computation for large values of $m\text{,}$ so we might more easily choose “random” values and check for the first value which is relatively prime to $m\text{.}$  The decryption exponent is the multiplicative inverse, mod $m\text{,}$ of the encryption exponent.  If you construct an improper encryption exponent (not relatively prime to $m$), the computation of the multiplicative inverse will fail (and Sage will tell you so).  We do this twice —- for both Alice and Bob.</p></div>

In [ ]:
factor(m_a)

In [ ]:
E_a = 5*23
D_a = inverse_mod(E_a, m_a)
D_a

In [ ]:
factor(m_b)

In [ ]:
E_b = 7*29
D_b = inverse_mod(E_b, m_b)
D_b

<div class="mathbook-content"><p id="p-1173">At this stage, each individual would publish their values of $n$ and $E\text{,}$ while keeping $D$ very private and secure.  In practice $D$ should be protected on the user's hard disk by a password only the owner knows.  For even greater security a person might only have two copies of their private key, one on a <abbr class="acronym">USB</abbr> memory stick they always carry with them, and a backup in their sage deposit box.  Every time the person uses $D$ they would need to provide the password.  The value of $m$ can be discarded.  For the record, here are all the keys:</p></div>

In [ ]:
print("Alice's public key, n:", n_a, "E:", E_a)

In [ ]:
print("Alice's private key, D:", D_a)

In [ ]:
print("Bob's public key, n:", n_b, "E:", E_b)

In [ ]:
print("Bob's private key, D:", D_b)

<div class="mathbook-content"><h3 class="heading hide-type" alt="Subsection  Signing and Encoding a Message"><span class="type">Subsection</span><span class="codenumber" /><span class="title">Signing and Encoding a Message</span></h3></div>

<div class="mathbook-content"><p id="p-1174">Alice is going to construct a message as an English word with four letters.  From these four letters we will construct a single number to represent the message in a form we can use in the <abbr class="acronym">RSA</abbr> algorithm.  The function <code class="code-inline tex2jax_ignore">ord()</code> will convert a single letter to its <abbr class="acronym">ASCII</abbr> code value, a number between 0 and 127.  If we use these numbers as “digits” mod 128, we can be sure that Alice's four-letter word will encode to an integer less than $128^4=268,435,456\text{.}$  The particular maximum value is not important, so long as it is smaller than our value of $n$ since all of our subsequent arithmetic is mod $n\text{.}$  We choose a popular four-letter word, convert to <abbr class="acronym">ASCII</abbr> “digits” with a list comprehension, and then construct the integer from the digits with the right base.  Notice how we can treat the word as a list and that the first digit in the list is in the “ones” place (we say the list is  in “little-endian” order).</p></div>

In [ ]:
word = 'Sage'
digits = [ord(letter) for letter in word]
digits

In [ ]:
message = ZZ(digits, 128)
message

<div class="mathbook-content"><p id="p-1175">First, Alice will sign her message to provide message verification.  She uses her private key for this, since this is an act that only she should be able to perform.</p></div>

In [ ]:
signed = power_mod(message, D_a, n_a)
signed

<div class="mathbook-content"><p id="p-1176">Then Alice encrypts her message so that only Bob can read it.  To do this, she uses Bob's public key.  Notice how she does not have to even know Bob — for example, she could have obtained Bob's public key off his web site or maybe Bob announced his public key in an advertisement in the <span class="booktitle">New York Times</span>.</p></div>

In [ ]:
encrypted = power_mod(signed, E_b, n_b)
encrypted

<div class="mathbook-content"><p id="p-1177">Alice's communication is now ready to travel on any communications network, no matter how insecure the network may be, and no matter how many snoops may be monitoring the network.</p></div>

<div class="mathbook-content"><h3 class="heading hide-type" alt="Subsection  Decoding and Verifying a Message"><span class="type">Subsection</span><span class="codenumber" /><span class="title">Decoding and Verifying a Message</span></h3></div>

<div class="mathbook-content"><p id="p-1178">Now assume that the value of <code class="code-inline tex2jax_ignore">encrypted</code> has reached Bob.  Realize that Bob may not know Alice, and realize that Bob does not even necessarily believe what he has received has genuinely originated from Alice.  An adversary could be trying to confuse Bob by sending messages that claim to be from Alice.  First, Bob must unwrap the encyption Alice has provided.  This is an act only Bob, as the intended recipient, should be able to do.  And he does it by using his private key, which only he knows, and which he has kept secure.</p></div>

In [ ]:
decrypted = power_mod(encrypted, D_b, n_b)
decrypted

<div class="mathbook-content"><p id="p-1179">Right now, this means very little to Bob.  Anybody could have sent him an encoded message.  However, this was a message Alice signed.  Lets unwrap the message signing.  Notice that this uses Alice's public key.  Bob does not need to know Alice — for example, he could obtain Alice's key off her web site or maybe Alice announced her public key in an advertisement in the <span class="booktitle">New York Times</span>.</p></div>

In [ ]:
received = power_mod(decrypted, E_a, n_a)
received

<div class="mathbook-content"><p id="p-1180">Bob needs to transform this integer representation back to a word with letters.  The <code class="code-inline tex2jax_ignore">chr()</code> function converts <abbr class="acronym">ASCII</abbr> code values to letters, and we use a list comprehension to do this repeatedly.</p></div>

In [ ]:
digits = received.digits(base=128)
letters = [chr(ascii) for ascii in digits]
letters

<div class="mathbook-content"><p id="p-1181">If we would like a slightly more recognizable result, we can combine the letters into a string.</p></div>

In [ ]:
''.join(letters)

<div class="mathbook-content"><p id="p-1182">Bob is pleased to obtain such an informative message from Alice.  What would have happened if an imposter had sent a message ostensibly from Alice, or what if an adversary had intercepted Alice's original message and replaced it with a tampered message?  (The latter is known as a “man in the middle” attack.)</p></div>

<div class="mathbook-content"><p id="p-1183">In either case, the rogue party would not be able to duplicate Alice's first action — signing her message.  If an adversary somehow signs the message, or tampers with it, the step where Bob unwraps the signing will lead to total garbage.  (Try it!)  Because Bob received a legitimate word, properly capitalized, he has confidence that the message he unsigned is the same as the message Alice signed.  In practice, if Alice sent several hundred words as her message, the odds that it will unsign as cohrent text are astronomically small.</p></div>

<div class="mathbook-content"><p id="p-1184">What have we demonstrated?</p></div>

<div class="mathbook-content"><ol class="decimal"><li id="li-296"><p id="p-1185">Alice can send messages that only Bob can read.</p></li><li id="li-297"><p id="p-1186">Bob can receive secret messages from anybody.</p></li><li id="li-298"><p id="p-1187">Alice can sign messages, so that then Bob (or anybody else)knows they are genuinely from Alice.</p></li></ol></div>

<div class="mathbook-content"><p id="p-1188">Of course, without making new keys, you can reverse the roles of Alice and Bob.  And if Carol makes a key pair, she can communicate with both Alice and Bob in the same fashion.</p></div>

<div class="mathbook-content"><p id="p-1189">If you want to use <abbr class="acronym">RSA</abbr> public-key encryption seriously, investigate the open source software GNU Privacy Guard, aka <code class="code-inline tex2jax_ignore">GPG</code>, which is freely available at <a class="url" href="https://www.gnupg.org/" target="_blank">www.gnupg.org/</a>.  Notice that it only makes sense to use encryption programs that allow you to look at the source code.</p></div>